# diff3words

Chirasree Mandal

December 13, 2021

In [1]:
import pandas as pd
import editdistance
import what3words 
import folium
import json

In [2]:
# Read in what3words API key
f = open('keys.json')
api_key = json.load(f)["api_key"]
f.close()

In [3]:
# Set up constants
w3w_red = "#CF3732"
dark_blue = "#0072BB"
yellow = "#e4cc37"
lt_blue = "#1e91d6"
green = "#8fc93a"

In [4]:
geocoder = what3words.Geocoder(api_key)

## First Example:

`///clip.apples.leaps` to `///dips.glue.fret`


In [5]:
# TODO: add docstring with assumptions
# Todo: add comments
def edit_distance_map(src, dest, steps):
    data = []
    for i in steps:
        res = geocoder.convert_to_coordinates(i)
        if res.get("country"):
            data.append([i, res["coordinates"]["lat"], res["coordinates"]["lng"], res["nearestPlace"], res["map"]])
            
    total_steps = len(data) - 1

    print("Source: ///{0}, lat: {1}, long: {2}, near {3}".format(src, data[0][1], data[0][2], data[0][3]))
    print("Dest: ///{0}, lat: {1}, long: {2}, near {3}".format(dest, data[total_steps][1], data[total_steps][2], data[total_steps][3]))


    print("Levenshtein Edit Distance: {}".format(editdistance.eval(src, dest)))

    df = pd.DataFrame(data, columns = ["words", "lat", "long", "nearestPlace", "url"])
    df_avg = df.mean().values
    
    m = folium.Map(location=df_avg, zoom_start=2, tiles="Stamen Toner", attr="")
    
    # Polyline Path
    # https://deparkes.co.uk/2016/06/03/plot-lines-in-folium/

    points = []
    for _, row in df.iterrows():
        points.append([row["lat"], row["long"]])
    folium.PolyLine(points, color=w3w_red, opacity=0.7).add_to(m)
    
    for index, row in df.iterrows():
        name = row["words"]
        url = row["url"]
        if name == src:
            colors = dark_blue
        elif name == dest:
            colors = yellow
        else:
            colors = w3w_red
        folium.RegularPolygonMarker(location=[row["lat"], row["long"]], number_of_sides=4, 
                                        rotation=45, radius=5, popup="Step {2}/{3} <a href={0}>///{1}</a>".format(url, name, index, total_steps), 
                                tooltip=None, color=colors, fill_color=colors, opacity=1.0).add_to(m)
    return m

In [6]:
src = "clip.apples.leaps"
dest = "dips.glue.fret"

In [7]:
# Steps
# https://phiresky.github.io/levenshtein-demo/
steps = [src, "dlip.apples.leaps",
"dip.apples.leaps",
"dips.apples.leaps",
"dips.gpples.leaps",
"dips.glples.leaps",
"dips.glules.leaps",
"dips.glues.leaps",
"dips.glue.leaps",
"dips.glue.feaps",
"dips.glue.freaps",
"dips.glue.fretps",
"dips.glue.frets",
"dips.glue.fret"]

In [8]:
m = edit_distance_map(src, dest, steps)
m

Source: ///clip.apples.leaps, lat: 38.975623, long: -75.811633, near Greensboro, Maryland
Dest: ///dips.glue.fret, lat: -25.585638, long: 115.0748, near Carnarvon, Western Australia
Levenshtein Edit Distance: 13


In [9]:
#m.save("clip-apples-leap-stamen-toner-map.html")

## Example 2 -- Nearby Squares

 `///desks.fish.sleepy` to `///gets.packet.text`

In [10]:
moffitt_library = "gets.packet.text"
mccone_hall = "desks.fish.sleepy"

steps2 = [mccone_hall, 
"gesks.fish.sleepy",
"getks.fish.sleepy",
"gets.fish.sleepy",
"gets.pish.sleepy",
"gets.pash.sleepy",
"gets.pach.sleepy",
"gets.pack.sleepy",
"gets.packesleepy",
"gets.packetleepy",
"gets.packet.eepy",
"gets.packet.tepy",
"gets.packet.texy",
"gets.packet.text"]

In [11]:
m2 = edit_distance_map(mccone_hall, moffitt_library, steps2)
m2

Source: ///desks.fish.sleepy, lat: 37.874178, long: -122.259682, near Berkeley, California
Dest: ///gets.packet.text, lat: 37.872372, long: -122.260878, near Berkeley, California
Levenshtein Edit Distance: 13


In [12]:
#m2.save("mccone-moffitt.html")